#### Relatório do projeto - Mineração de Dados

##  Autores

## 1. Introdução

Este trabalho estuda técnicas de mineração de dados para classificar áudio. Em específico, o universo a ser classificado consiste em dez caracteres: 6, 7, a, b, c, d, h, m, n, x. Cada arquivo de áudio, chamado de _captcha_, contém 4 caracteres, podendo haver caracteres repetidos ou não. O objetivo é identificar quais caracteres estão presentes nos _captchas_. A classificação de um _captcha_ é considerada sucesso quando **todos** os seus quatro caracteres são corretamente identificados. Caso haja uma ou mais caracteres erroneamente classificados em um _captcha_, a classificação é considerada errada para aquele _captcha_.

### Ferramentas necessárias para a reprodução:

Para este trabalho foram utilizadas as ferramentas mencionadas abaixo. É importante prestar atenção nas versões das mesmas, quando indicadas.

#### Programas e ambientes
- FFMPEG
- Anaconda (Python>=3.5) 
- Linux

#### Bibliotecas python
-  numpy>=1.13.3
-  pandas>=0.20.3
-  scikit-learn>=0.19.1
-  scipy==1.1.0 
-  librosa>=0.6.1
-  matplotlib>=1.5.3
-  Sox (`conda install -c conda-forge sox`)

#### Pastas
- É assumido que no diretório em que se encontra este documento haja uma pasta chamada *fase\_1*. Dentro desta pasta deve haver duas pastas: *base\_treinamento\_I* e *base\_validacao\_I*, cada uma uma com os captchas de áudio.


Numpy e Pandas são bibliotecas criadas para manipular vetores de uma forma mais otimizada e rica em informação em relação à classe _list_ da biblioteca padrão do Python. Scikit-learn é uma biblioteca que implementa diversos algoritmos de mineração de dados, bem como técnicas de transformação e pré-processamento. Scipy é uma biblioteca usada para computação científica. Matplotlib fornece funções para plotar imagens e gráficos. Por fim, librosa é uma biblioteca para manipulação de áudio. 

Além das bibliotecas citadas, já fornecemos junto com os arquivos a biblioteca `pysndfx`, que nada mais é do que um *wrapper* para o Sox. A biblioteca está disponível no [Github](https://github.com/carlthome/python-audio-effects).

## 2. Análise exploratória

A primeira tarefa é segmentar os _captchas_. Como cada _captcha_ contém quatro caracteres, a ideia é gerar quatro arquivos .wav, um para cada caractere.

In [ ]:
import sys
import os

sys.path.append(os.path.abspath('..'))

In [ ]:
#import librosa
#import matplotlib.pyplot as plt
#import librosa.display
import numpy as np
#from pysndfx import AudioEffectsChain
#from scipy.signal import find_peaks

In [ ]:
import trim_peaks as trim

trim.trim('./fase_1/base_treinamento_I/abn7.wav', '', True) #desenha o grafico
#em azul, o audio original
#em laranja,o audio processado
#os riscos vermelhos são os pontos finais

A segmentação segue os passos:

- O áudio é normalizado (tornando o maior pico = 1) e os silêncios ao início e ao fim são removidos;
- Um `limiter` é aplicado, com o parâmetro de 20dB. Isso significa que qualquer sinal com intensidade maior que este limite são diminuídos até 20dB. Isso é feito para tentar reduzir a diferença entre picos mais altos e mais baixos, pois nem todas as letras são gravadas com a mesma intensidade;
- Frequências acima de 2500Hz e abaixo de 100Hz são descartadas, pois são frequências com pouca informação vocal;
- Então, um equalizador é aplicado na faixa de 300Hz, intensificando essas frequências por 15dB. Na prática, isso causa um *boost* nas regiões onde há grande informação vocal, potencialmente até distorcendo-as, e deixando a diferença entre voz e fundo muito mais clara;
- Samples com intensidade menor que 0.5 são zerados, em uma tentativa de remover ruído de gravação. Idealmente, regiões silenciosas (sem voz) seriam zeradas;
- A função `find_peaks`, da biblioteca `scipy.signal` é usada para encontrar os picos do áudio. Essa função retorna todos os máximos e mínimos locais de um sinal. Mesmo passando parâmetros de altura mínima (de forma a apenas encontrar máximos) e distância mínima de 1s entre picos, por vezes a função encontra mais do que 4 picos, chamados daqui em diante de pontos;
- A intenção é que cada ponto represente um caractere. São escolhidos os pares de pontos mais próximos um do outro, e ambos são removidos e substituídos pela sua média. Isso é feito até que restem apenas 4 pontos. Teoricamente, ao fazer isso estamos pegando os caracteres com mais de um pico e colapsando suas posições até restar apenas um ponto, próximo ao seu centro;
- A partir de cada ponto encontrado, é extraída do áudio original uma região de 1s para cada lado, totalizando no máximo 2s de áudio. O ruído de fundo (silêncio) de cada lado e removido, isolando apenas a fala, e, como essa operação normalmente remove o começo e final do caractere, 0.25s são adicionados de volta de cada lado;
- Os áudios encontrados são exportados, assumindo que foram encontrados na ordem em que estão presentes no nome do arquivo.


Em um primeiro momento, é necessário criar a estrutura de pastas para que os algoritmos de mineração de dados atuem. É **assumido** que haja, no atual diretório, uma pasta chamada *fase\_1/base\_treinamento\_I* e uma pasta chamada *fase\_1\_base_validacao\_I*. Dentro dessas pastas, há uma pasta para cada _captcha_ contendo um arquivo .wav, cujo nome são os quatro caracteres que compõem o _captcha_ na ordem em que aparecem.

In [ ]:
from main import create_folder_structure
create_folder_structure()

Desta forma, a estrutura de pastas necessária para a classificação é montada. Os arquivos segmentados ficam nas pastas *output\_training* e *output\_test*. Dentro de cada pasta há um arquivo .wav para cada caractere identificado na segmentação. O nome de cada arquivo é *ordem\-caractere.wav*. Por exemplo, se o terceiro caractere de um dado _captcha_ for a letra "d", o nome do arquivo correspondente a ele será *3-d.wav*.

In [ ]:
import classification_test

A seguir é necessário extrair atributos da base de dados.

In [ ]:
from spectrum import get_spectrum

A função `get_spectrum` extrai a intensidade média dentro de cada um dos [espectros de áudio](https://www.teachmeaudio.com/mixing/techniques/audio-spectrum/). Por exemplo, os espectros mais agudos poderiam ajudar um classificador a separar caracteres sibilantes (com sons de "S", como x e 7) de caracteres não sibilantes (como m e h).

O treinamento é feito sobre toda a base de treino, e os dados são normalizados de forma a terem média 0 e desvio 1. Isso é importante principalmente para algoritmos baseados em distância, de forma que atributos com valores maiores não dominem a classificação em detrimento de outros cujos valores tendem a ser mais próximos de zero.

In [ ]:
corretos_svm = {'6':0, '7':0, 'a':0, 'b':0, 'c':0, 'd':0, 'h':0, 'm':0, 'n':0, 'x':0}
corretos_1nn = {'6':0, '7':0, 'a':0, 'b':0, 'c':0, 'd':0, 'h':0, 'm':0, 'n':0, 'x':0}
errados_svm  = {'6':0, '7':0, 'a':0, 'b':0, 'c':0, 'd':0, 'h':0, 'm':0, 'n':0, 'x':0}
errados_1nn  = {'6':0, '7':0, 'a':0, 'b':0, 'c':0, 'd':0, 'h':0, 'm':0, 'n':0, 'x':0}
elementos    = {'6':0, '7':0, 'a':0, 'b':0, 'c':0, 'd':0, 'h':0, 'm':0, 'n':0, 'x':0}

number_of_folders = len(classification_test.TEST_AUDIO_CAPTCHA_FOLDERS)
number_of_characters = len(classification_test.TEST_AUDIO_FILENAMES)

In [ ]:
def preliminar_test(X_train: np.ndarray, y_train: np.ndarray, std_scale: preprocessing.data.StandardScaler):
    accuracy1NN = 0
    accuracySVM = 0
    total1NN = 0
    totalSVM = 0
    for folder in classification_test.TEST_AUDIO_CAPTCHA_FOLDERS:
        correct1NN = 0
        correctSVM = 0
        for filename in os.listdir(folder):
            obj = extract_features(filename, folder)
            y_test = obj[obj.size - 1]
            X_test_raw = [obj[0:obj.size - 1]]
            X_test = std_scale.transform(X_test_raw) # Normalisar
            
            elementos[y_test] += 1
            
            neigh1 = KNeighborsClassifier(n_neighbors=1)
            y_pred = neigh1.fit(X_train, y_train).predict(X_test)
            if y_pred[0] == y_test:
                correct1NN+=1
                total1NN+=1
                corretos_1nn[y_test] += 1
            else:
                errados_1nn[y_test] += 1

            clf = SVC()
            y_pred = clf.fit(X_train, y_train).predict(X_test)
            if y_pred[0] == y_test:
                correctSVM+=1
                totalSVM+=1
                corretos_svm[y_test] += 1
            else:
                errados_svm[y_test] += 1

        if correct1NN == 4:
            accuracy1NN+=1
        if correctSVM == 4:
            accuracySVM+=1

    print("Acuracia SVM (captcha)    = {0:.2f}%".format((accuracySVM / number_of_folders)*100))
    print("Acuracia SVM (caracteres) = {0:.2f}%".format((totalSVM / number_of_characters)*100))
    print("Acuracia 1NN (captcha)    = {0:.2f}%".format((accuracy1NN / number_of_folders)*100))
    print("Acuracia 1NN (caracteres) = {0:.2f}%".format((total1NN / number_of_characters)*100))
    print("RESULTADOS SVM")
    print("Caracter    Acertos          Taxa")
    for k, v in corretos_svm.items():
        print(k,"         ",v,"/",elementos[k], "        ", "{0:.2f}%".format((v/elementos[k])*100))
    print("Caracter    Erros            Taxa")
    for k, v in errados_svm.items():
        print(k,"         ",v,"/",elementos[k], "        ", "{0:.2f}%".format((v/elementos[k])*100))
        
    print("")
    print("RESULTADOS 1NN")
    print("Caracter    Acertos          Taxa")
    for k, v in corretos_1nn.items():
        print(k,"         ",v,"/",elementos[k], "        ", "{0:.2f}%".format((v/elementos[k])*100))
    print("Caracter    Erros            Taxa")
    for k, v in errados_1nn.items():
        print(k,"         ",v,"/",elementos[k], "        ", "{0:.2f}%".format((v/elementos[k])*100))

In [ ]:
X_train, y_train, std_scale = classification_test.train()

In [ ]:
preliminar_test(X_train, y_train, std_scale)

| Métrica                    | Taxa  |
|:-:|:-:|
|Acuracia SVM (captcha)      | 25.17%|
|Acuracia SVM (caracteres)   | 69.31%|
|Acuracia 1NN (captcha)      | 22.45%|
|Acuracia 1NN (caracteres)   | 67.07%|


**Resultados SVM**

|Caracter| Acerto        | Erro          |
|:--------|---------------:|:---------------|
|6       |41/65 (63.08%) |24/65 (36.92%) |
|7       |58/61 (95.08%) |03/61 (04.92%) |
|a       |50/61 (81.97%) |11/61 (18.03%) |
|b       |48/58 (82.76%) |10/58 (17.24%) |
|c       |44/67 (65.67%) |23/67 (34.33%) |
|d       |04/47 (08.51%) |43/47 (91.49%) |
|h       |62/68 (91.18%) |06/68 (08.82%) |
|m       |26/62 (41.94%) |36/62 (58.06%) |
|n       |36/47 (76.60%) |11/47 (23.40%) |
|x       |33/44 (75.00%) |11/44 (25.00%) |


**Resultados 1NN**

|Caracter| Acerto        | Erro          |
|:--------|---------------:|:---------------|
|6       |40/65 (61.54%) |25/65 (38.46%) |
|7       |52/61 (85.25%) |09/61 (14.75%) |
|a       |41/61 (67.21%) |20/61 (32.79%) |
|b       |34/58 (58.62%) |24/58 (41.38%) |
|c       |45/67 (67.16%) |22/67 (32.84%) |
|d       |18/47 (38.30%) |29/47 (61.70%) |
|h       |59/68 (86.76%) |09/68 (13.24%) |
|m       |39/62 (62.90%) |23/62 (37.10%) |
|n       |30/47 (63.83%) |17/47 (36.17%) |
|x       |31/44 (70.45%) |13/44 (29.55%) |

A partir de uma análise preliminar, foi percebido que o algoritmo SVM apresentava, no geral, bons índices de classificação. Contudo, sua taxa de erro era superior à de acerto para os caracteres 'd' e 'm'. Contudo, o algoritmo 1NN apresentava resultados superiores para este caractere. 

## 3. Metodologia

Para cada caso, é calculada a acurácia na detecção de cada caracter. Também é calculada a acurácia geral na detecção de caracteres. Por fim, é calculada a acurácia na detecção de _captchas_.

Os algoritmos usados, conforme já mencionado, são 1NN e SVM.

- 1NN:

    Uma variante do K-Nearest Neighbour (KNN) quando K=1. Em resumo, ele recebe um elemento de teste e calcula sua distância para os demais elementos na base de dados. O elemento de teste recebe a classe do elemento mais próximo. Podem ser usadas diferentes métricas de distância. As mais comuns são as distâncias euclidianas, Manhattan e Mahalanobis. No presente caso, foi usada a distância euclidiana.


- SVM:

    Sigla para _Support Vector Machine_. A partir de uma base de dados com elementos rotulados, o algoritmo constrói um modelo de pontos em um espaço de forma que cada classe esteja no espaço mais amplo possível. Então, ele traça retas, e os elementos são classificados conforme o local em que são colocados.

In [ ]:
errados   = {'6':0, '7':0, 'a':0, 'b':0, 'c':0, 'd':0, 'h':0, 'm':0, 'n':0, 'x':0}
corretos  = {'6':0, '7':0, 'a':0, 'b':0, 'c':0, 'd':0, 'h':0, 'm':0, 'n':0, 'x':0}
elementos = {'6':0, '7':0, 'a':0, 'b':0, 'c':0, 'd':0, 'h':0, 'm':0, 'n':0, 'x':0}

A análise exploratória mostrou que o algoritmo SVM apresentava um bom desempenho geral, mas nos casos específicos das letras 'd' e 'm', sua taxa de acerto era inferior à taxa de erro. O 1NN, por outro lado, apresentava resultados melhores. 

Foi usada, portanto, a seguinte heurística: o resultado escolhido é aquele indicado pelo SVM. Porém, caso tal resultado seja os caracteres 'd' ou 'm', opta-se pela resposta de 1NN. Como em somente 8% dos casos o SVM reportava 'd' como resposta, tal abordagem resultaria em um recall muito baixo da letra 'd'. Então, também foi decidido que sempre que 1NN reportasse o caractere 'd' ou o caractere 'm', sua resposta seria a escolhida.

## 4. Resultados

In [ ]:
classification_test.test(X_train, y_train, std_scale)

**Resultados finais**

|Caracter| Acerto        | Erro          |
|:--------|---------------:|:---------------|
|6       |41 / 65 (63.08%) |24/65 (36.92%) |
|7       |58 / 61 (95.08%) |03/61 (04.92%) |
|a       |50 / 61 (81.97%) |11/61 (18.03%) |
|b       |36 / 58 (62.07%) |22/58 (37.93%) |
|c       |45 / 67 (67.16%) |22/67 (32.84%) |
|d       |18 / 47 (38.30%) |29/47 (61.70%) |
|h       |61 / 68 (89.71%) |07/68 (10.29%) |
|m       |39 / 62 (62.90%) |23/62 (37.10%) |
|n       |31 / 47 (65.96%) |16/47 (34.04%) |
|x       |33 / 44 (75.00%) |11/44 (25.00%) |

|Métrica| Taxa        |
|---------------:|:---------------|
|Acuracia (captcha) |  27.89%|
|Acuracia (caracteres) | 71.03%|

A acurácia obtida na detecção de _captchas_ está condizente com a acurácia individual de detecção de caracteres. A afirmação vem da suposição de que um classificador que obtenha 71% de acurácia para os caracteres tende, ao classificar um _captcha_ com quatro caracteres, obteria: 

$$0.71 \cdot 0.71 \cdot 0.71 \cdot 0.71 = 0.71^4 = 25.4\%$$

Nossos testes chegaram a 27.89% de acerto, próximo da expectativa teórica.

## 5. Comentários finais

- Dificuldades encontradas

    A segmentação não foi um processo trivial. Separar o áudio em faixas de tempo pré-determinadas era uma estratégia perigosa, e por isso foi necessário elaborar uma estratégia mais refinada.
    
    O tempo de execução dos diversos passos descritos acima, que somados demoram algo na casa dos minutos, também se mostrou uma dificuldade prática na hora de avaliar os algoritmos.


- Ideias que não foram exploradas e a razão